# Database (db) creator and ID trasnsfer to input file


This notebook loads excel files and combines them into a database

In [2]:
# Import packages
import os
from glob import glob
import pandas as pd
import fafbseg

In [3]:
# Some custom functions

def update_dataframe_single_column(source_df, target_df, reference_column):
    # Create a dictionary mapping from the reference column to the source DataFrame
    reference_dict = source_df.groupby(reference_column).first().reset_index().to_dict(orient='records')
    reference_dict = {row[reference_column]: row for row in reference_dict}

    # Update the target DataFrame based on the reference column
    for i, row in target_df.iterrows():
        ref = row[reference_column]
        if ref in reference_dict:
            source_row = reference_dict[ref]
            target_df.loc[i] = source_row

    return target_df

### 1. Loading all data sets of interest in a loop

The original excell files need tzo be stored in the same folder. All files in that folder will be loaded


In [4]:
# Data paths
# Choose path and file
PC_disc = 'D' # why is this necessary? MI
dataPath = f'{PC_disc}:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database' #\Connectomics-Data\FlyWire\Excels\drive-data-sets\database'  #D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database
# remember that f'{} are used for string formatting in python. {PC_disc} is a place holder. I can use that to enter varibales in a string! eg f"My name is {name} and I am {age} years old."
# To use a raw string 'r'\blahblah'
fileName_ls = glob(dataPath +"\\"+ "*.xlsx") # adds all file names from the specific direcotry with the xslx extention. \\ path seperator for windows systems

#Creating the database in a loop
df_ls = [] # empty list
for fileName in fileName_ls:
    print(f'Importing: {fileName}')
    filePath = os.path.join(dataPath,fileName)
    df = pd.read_excel(filePath)
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str) ??MI
    if df["seg_id"][0] == 'asdf': 
        df = df.iloc[1: , :] # removes the first row. Why?
        df.reset_index(inplace=True,drop=True)
    df_ls.append(df) # a list of dataframes

Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\C2 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\C3 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm1 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm10 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm12 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm15 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm2 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm4 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm6 proofreadings.xlsx
Importing

### 2. Creating, updating and filtering the database (db) of interest

In [5]:
### Creation
db = pd.concat(df_ls) # new df with concatenated all dfs
print(f'\n\nCell types in the database: {db.symbol.unique()}, total = {len(db.symbol.unique())}') # get the unique neurons, \n\n just two spaced lines when printing
# how does he put the percent scale bar in the printing area?
## Removing any Nan columns
db = db[db["Updated_seg_id"].notna()] # why would there be any nans there?

### Filtering and updating database
## Chossing optic lobe of interest:
_hemisphere = 'R' # variable names can start with an underscore 
db_R = db[db.hemisphere != 'L'].copy() # not equal to L, copies dataframe without the L

## Updating segmnet ids
#Getting the lists of IDs to update
curr_ID_ls = db_R["Updated_seg_id"].tolist() # df to list
curr_ID_ls = db_R["seg_id"].tolist() # isn't he rewriting here?
#Updating all IDs at once
updated_ID_df = fafbseg.flywire.update_ids(curr_ID_ls, stop_layer=2, supervoxels=None, timestamp=None, 
                                           dataset='production', progress=True)
db_R['Updated_seg_id'] = updated_ID_df['new_id'].astype(str).tolist() # astype(str) converts to string 
db_R['Updated_confidence'] = updated_ID_df['confidence'].astype(str).tolist() # what is confidence?



Cell types in the database: ['C2' 'C3' 'Dm1' 'Dm10' 'Dm12' 'Dm15' 'Dm2a' 'Dm2b' 'Dm2c' 'Dm4' 'Dm6'
 'L1' 'L2' 'L3' 'L5' 'Me-Lo-2-N.I.' 'Mi1' 'Mi10' 'Mi13' 'Mi4' 'Mi9' 'ML1'
 'Pm1_Tm1' 'Pm1_Tm2' 'Pm1a_Tm1' 'Pm1a_Mi1' 'Pm2' 'Pm3' 'Pm4' 'OA-AL2b2-L1'
 'OA-AL2b2-L2' 'OA-AL2b2-R1' 'OA-AL2b2-R2' 'CT1' 'pMP7' 'PS125' 'R7' 'R8'
 'T1' 'T3' 'T4a' 'T4b' 'T4c' 'T4d' 'Tm1' 'Tm1/2' 'Tm16' 'Tm2' 'Tm20'
 'Tm20?' 'Tm3' 'Tm4' 'Tm4a' 'Tm9' 'Tm10' 'Tm11' 'Tm12' 'TmY10' 'TmY15'
 'TmY17'], total = 60


Updating:   0%|          | 0/21022 [00:00<?, ?it/s]

In [6]:
db_R

,XYZ-ME,symbol,XYZ-LO,quality comment,hemisphere,seg_id,identified_in,Updated_seg_id,Update_confidence,updated_seg_id,...,lab authorship (Y/N),patch_id,column_id,dark_neurons (Y/N),bad_quality_rim (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),Updated_confidence,Mi4-connected,dorso-ventral
250,NaN,C2,NaN,strange dendrite,R,720575940649255801,"Fischbach & Dittrich, 1989",720575940647498019,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
251,NaN,C2,NaN,strange dendrite,R,720575940617134678,"Fischbach & Dittrich, 1989",720575940625965756,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,NaN,NaN
252,NaN,C2,NaN,strange dendrite,R,720575940616540043,"Fischbach & Dittrich, 1989",720575940616540043,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
253,NaN,C2,NaN,no axon terminals,R,720575940612496497,"Fischbach & Dittrich, 1989",720575940612496497,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
254,NaN,C2,NaN,strange dendrite,R,720575940618589953,"Fischbach & Dittrich, 1989",720575940618589953,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,NaN,TmY17,NaN,NaN,R,720575940633505934,NaN,720575940633505934,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
152,NaN,TmY17,NaN,NaN,R,720575940651270902,NaN,720575940651270902,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
153,NaN,TmY17,NaN,NaN,R,720575940625696637,NaN,720575940625696637,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
154,NaN,TmY17,NaN,NaN,R,720575940625313428,NaN,720575940625313428,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [20]:
db_R['symbol'].unique()
db_R

,XYZ-ME,symbol,XYZ-LO,quality comment,hemisphere,seg_id,identified_in,Updated_seg_id,Update_confidence,updated_seg_id,...,patch_id,column_id,dark_neurons (Y/N),bad_quality_rim (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),Updated_confidence,Mi4-connected,dorso-ventral,Updated_seg_ids
250,NaN,C2,NaN,strange dendrite,R,720575940649255801,"Fischbach & Dittrich, 1989",720575940647498019,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940647498019
251,NaN,C2,NaN,strange dendrite,R,720575940617134678,"Fischbach & Dittrich, 1989",720575940625965756,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.99,NaN,NaN,720575940625965756
252,NaN,C2,NaN,strange dendrite,R,720575940616540043,"Fischbach & Dittrich, 1989",720575940616540043,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940616540043
253,NaN,C2,NaN,no axon terminals,R,720575940612496497,"Fischbach & Dittrich, 1989",720575940612496497,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940612496497
254,NaN,C2,NaN,strange dendrite,R,720575940618589953,"Fischbach & Dittrich, 1989",720575940618589953,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940618589953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,NaN,TmY17,NaN,NaN,R,720575940633505934,NaN,720575940633505934,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940633505934
152,NaN,TmY17,NaN,NaN,R,720575940651270902,NaN,720575940651270902,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940651270902
153,NaN,TmY17,NaN,NaN,R,720575940625696637,NaN,720575940625696637,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940625696637
154,NaN,TmY17,NaN,NaN,R,720575940625313428,NaN,720575940625313428,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940625313428


### 3. Creating a input file of interest

In [46]:
#Selecting postsynaptic neuron of interest
neuron = 'Mi1'
neuron_df = db_R[db_R['symbol'] == neuron].copy()

#Filtering for valid segment ids based on a given criteria
neuron_selected_df = neuron_df.copy()
neuron_selected_df = neuron_selected_df.copy() # but I will just take all MI
# neuron_selected_df[neuron_selected_df['backbone proofread (Y/N)'] == 'Y'].copy()
neuron_selected_df = neuron_selected_df.copy() # Temp just for C3 neuron_selected_df.iloc[:135].copy()
#neuron_selected_df = neuron_selected_df[neuron_selected_df['inputs_proofread (Y/N)'] == 'N'].copy()

print(f'\n\nTotal number of postsynaptic cells: {len(neuron_selected_df)}\n\n')

ID_ls = neuron_selected_df['Updated_seg_id'].tolist()

####temp_maria
#Mi1s_patch1 = [720575940621959185,720575940610177941,720575940621647996,720575940636894022,720575940610254037,720575940639404605,720575940629418110,720575940603042272,720575940650403449,720575940617839832,720575940615088034,720575940632717359,720575940630171207,720575940644391112,720575940625269822,720575940609463620]
#Mi1s_patch2 = [720575940611524835,720575940631147484,720575940610460402,720575940630964551,720575940610772174,720575940610674090,720575940612556438,720575940637260133,720575940629763663,720575940610953478,720575940616367133,720575940629678124,720575940632107033,720575940628892432,720575940610952198,720575940614282134]
#Mi1s_patch3 = [720575940634823822,720575940615420358,720575940631379512,720575940624234644,720575940632700256,720575940633027539,720575940624177900,720575940622954189,720575940625383944,720575940627733447,720575940621505718,720575940626810442,720575940623328184,720575940627160143,720575940628618920,720575940637643950]
Mi1s_patch4 = [720575940640572733,720575940622304895,720575940630827339,720575940639250741,720575940613120741,720575940636425870,720575940619977087,720575940631894753,720575940622933444,720575940631103007,720575940626104648,720575940609555406,720575940628157595,720575940627970122,720575940626900086,720575940627669239]
#Mi1s_patch5 = [720575940627216484,720575940622490381,720575940632807359,720575940623530032,720575940636962138,720575940620298754,720575940611751347,720575940634978222,720575940604191456,720575940629426450,720575940634024303,720575940629855964,720575940617029009,720575940618338260,720575940610740394,720575940617578267]
#Mi1s_patch6 = [720575940644945352,720575940634766699,720575940633138330,720575940606398240,720575940625273118,720575940630386060,720575940644950216,720575940622977791,720575940614503285,720575940629090048,720575940614935249,720575940632113439,720575940623942408,720575940623628105,720575940626684689,720575940616572992,720575940629829275,720575940660975745,720575940643266583,720575940605262880,720575940632443804,720575940632238675,720575940629262922,720575940620500801,720575940632811112,720575940646369588,720575940613508210,720575940610952982,720575940616553541,720575940630050920,720575940627088330,720575940639131876,720575940612410481,720575940628637672,720575940610578786,720575940623666320]
#Mi1s_patch7 = [720575940632856289,720575940613994898,720575940616304925,720575940628314643,720575940637746493,720575940625924995,720575940630000399,720575940628273468,720575940607730780,720575940630012923,720575940606944898,720575940608810798,720575940630046736,720575940628196908,720575940622893150,720575940632845463]
Mi1s =Mi1s_patch4

####temp_maria_end
#Fetching the neuron's inputs and putputs
neurons_inputs = fafbseg.flywire.synapses.fetch_synapses(Mi1s, pre=False, post=True, attach=True,  # Mi1s # ID_ls
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "latest")

neurons_outputs = fafbseg.flywire.synapses.fetch_synapses(Mi1s, pre=True, post=False, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "latest") # here seb had mat = "live" but produces an error to me possibly from the cave version


#Counting inputs per ID, option joining dataframes
final_input_df = pd.DataFrame()
for n in neurons_inputs['post'].unique():
    inputs_count = {} # creates an empty dictionary
    curr_inputs = neurons_inputs[neurons_inputs['post'] == n]
    inputs_str = curr_inputs.applymap(str)
    
    for c in inputs_str['pre'].to_list():
        inputs_count[c] = inputs_count.get(c, 0) + 1 # access the dictionary with c as key
    input_count_df = pd.DataFrame(inputs_count, index=[0])
    input_count_df = input_count_df.T
    input_count_df.rename(columns={0: "counts"},inplace=True)
    input_count_df.index.names = ['presynaptic_ID']
    input_count_df = input_count_df.sort_values(by="counts",ascending=False)
    input_count_df['postsynaptic_ID'] = inputs_str['post'].to_list()[0:len(input_count_df)]
    final_input_df = final_input_df.append(input_count_df)
    #print(f'Counting done for: {n}')
input_count_str_df = final_input_df.applymap(str)
print('INPUTS: ')
display(input_count_str_df.head())


#Counting outputs per ID, option joining dataframes
final_output_df = pd.DataFrame()
for n in neurons_outputs['pre'].unique():
    outputs_count = {}
    curr_outputs = neurons_outputs[neurons_outputs['pre'] == n]
    outputs_str = curr_outputs.applymap(str)
    
    for c in outputs_str['post'].to_list():
        outputs_count[c] = outputs_count.get(c, 0) + 1
    output_count_df = pd.DataFrame(outputs_count, index=[0])
    output_count_df = output_count_df.T
    output_count_df.rename(columns={0: "counts"},inplace=True)
    output_count_df.index.names = ['postsynaptic_ID']
    output_count_df = output_count_df.sort_values(by="counts",ascending=False)
    output_count_df['presynaptic_ID'] = outputs_str['pre'].to_list()[0:len(output_count_df)]
    final_output_df = final_output_df.append(output_count_df)
    #print(f'Counting done for: {n}')
output_count_str_df = final_output_df.applymap(str)
print('OUTPUTS: ')
display(output_count_str_df.head())




Total number of postsynaptic cells: 797




Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


INPUTS: 


,counts,postsynaptic_ID
presynaptic_ID,,
720575940627244165,164,720575940639250741
720575940619166518,62,720575940639250741
720575940627317002,39,720575940639250741
720575940633657631,24,720575940639250741
720575940625500734,20,720575940639250741


OUTPUTS: 


,counts,presynaptic_ID
postsynaptic_ID,,
720575940639071721,84,720575940622304895
720575940618947329,47,720575940622304895
720575940619319454,44,720575940622304895
720575940633594669,44,720575940622304895
720575940627351913,44,720575940622304895


### 4. Adding useful information to the data frame

In [47]:
## For INPUTS


# Selecting dataframe
#Updating the IDs via Fafbseg
partner_ID = input_count_str_df.index.tolist()
updated_ID_df = fafbseg.flywire.update_ids(partner_ID, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
partner_ID_ls = updated_ID_df["new_id"].tolist()

# Identifying user-based annotations about cell identity

identification_df = fafbseg.flywire.find_celltypes(partner_ID_ls, user=None, exact=False, case=False, regex=True, update_roots=False)
identification_no_duplicates_df = identification_df.drop_duplicates(subset='pt_root_id', keep='last', inplace=False, ignore_index=False).copy()

# Adding info to the current data set

# Selecting dataframes and resetting index
source_df = identification_no_duplicates_df.copy()
source_df.reset_index(inplace = True, drop = True)
target_df = input_count_str_df.copy()
target_df.reset_index(inplace = True, drop = True)


# Adding columns for the function to properly work
target_df['presynaptic_ID'] = input_count_str_df.index.astype(str)
source_df['presynaptic_ID'] = identification_no_duplicates_df['pt_root_id'].tolist()
target_df['guess'] = None
source_df['guess'] = identification_no_duplicates_df['tag'].tolist()
target_df['author'] = None
source_df['author'] = identification_no_duplicates_df['user_id'].tolist()

# Function inputs
source_cols = ['guess', 'author','presynaptic_ID']
target_cols = ['guess', 'author', 'presynaptic_ID']
reference_column = 'presynaptic_ID'

source_df = source_df[source_cols].copy()
target_df = target_df[source_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)


# Running the function and completing the dataset
result_input_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_input_df['counts'] = input_count_str_df['counts'].tolist()
result_input_df['postsynaptic_ID'] = input_count_str_df['postsynaptic_ID'].tolist()
result_input_df.head()

Updating:   0%|          | 0/1471 [00:00<?, ?it/s]

,guess,author,presynaptic_ID,counts,postsynaptic_ID
0,L1,94,720575940627244165,164,720575940639250741
1,L5,94,720575940619166518,62,720575940639250741
2,L3,94,720575940627317002,39,720575940639250741
3,"OA-AL2b2; Busch et al., 2009; Tdc2-Gal4-positi...",2395,720575940633657631,24,720575940639250741
4,Proximal medulla 2; Pm2,17,720575940625500734,20,720575940639250741


In [48]:
## For OUTPUTS

# Selecting dataframe
#Updating the IDs via Fafbseg
partner_ID = output_count_str_df.index.tolist()
updated_ID_df = fafbseg.flywire.update_ids(partner_ID, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
partner_ID_ls = updated_ID_df["new_id"].tolist()

# Identifying user-based annotations about cell identity

identification_df = fafbseg.flywire.find_celltypes(partner_ID_ls, user=None, exact=False, case=False, regex=True, update_roots=False)
identification_no_duplicates_df = identification_df.drop_duplicates(subset='pt_root_id', keep='last', inplace=False, ignore_index=False).copy()

# Adding info to the current data set

# Selecting dataframes and resetting index
source_df = identification_no_duplicates_df.copy()
source_df.reset_index(inplace = True, drop = True)
target_df = output_count_str_df.copy()
target_df.reset_index(inplace = True, drop = True)


# Adding columns for the function to properly work
target_df['postsynaptic_ID'] = output_count_str_df.index.astype(str)
source_df['postsynaptic_ID'] = identification_no_duplicates_df['pt_root_id'].tolist()
target_df['guess'] = None
source_df['guess'] = identification_no_duplicates_df['tag'].tolist()
target_df['author'] = None
source_df['author'] = identification_no_duplicates_df['user_id'].tolist()

# Function inputs
source_cols = ['guess', 'author','postsynaptic_ID']
target_cols = ['guess', 'author', 'postsynaptic_ID']
reference_column = 'postsynaptic_ID'

source_df = source_df[source_cols].copy()
target_df = target_df[source_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)


# Running the function and compleating the dataset
result_output_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_output_df['counts'] = output_count_str_df['counts'].tolist()
result_output_df['presynaptic_ID'] = output_count_str_df['presynaptic_ID'].tolist()
result_output_df.head()

Updating:   0%|          | 0/9582 [00:00<?, ?it/s]

,guess,author,postsynaptic_ID,counts,presynaptic_ID
0,Pm4; Proximal medullary amacrine neuron 4; FBb...,392,720575940639071721,84,720575940622304895
1,Proximal medulla 2; Pm2,17,720575940618947329,47,720575940622304895
2,"Tm3; Transmedullary neuron 3; Fischbach, K.F.,...",392,720575940619319454,44,720575940622304895
3,Proximal medulla 1a; Pm1a,17,720575940633594669,44,720575940622304895
4,Proximal medulla 1a; Pm1a,17,720575940627351913,44,720575940622304895


### 5. Transfering information from the main database (db)

In [49]:
## For INPUTS

# Matcing data types
db_R['Updated_seg_id'] = db_R['Updated_seg_id'].astype(str)
result_input_df['presynaptic_ID'] = result_input_df['presynaptic_ID'].astype(str)

# Merging the DataFrames based on common values
merged_input_df = pd.merge(result_input_df, db_R[['Updated_seg_id', 'symbol']], left_on='presynaptic_ID', right_on='Updated_seg_id', how='left')

# Drop the extra 'seg_id' column
merged_input_df.drop(columns=['Updated_seg_id'], inplace=True)

# Display the merged DataFrame
print('For INPUTS')
display(merged_input_df)


## For OUTPUTS

# Matcing data types
db_R['Updated_seg_id'] = db_R['Updated_seg_id'].astype(str)
result_output_df['postynaptic_ID'] = result_output_df['postsynaptic_ID'].astype(str)

# Merging the DataFrames based on common values
merged_output_df = pd.merge(result_output_df, db_R[['Updated_seg_id', 'symbol']], left_on='postsynaptic_ID', right_on='Updated_seg_id', how='left')

# Drop the extra 'seg_id' column
merged_output_df.drop(columns=['Updated_seg_id'], inplace=True)

# Display the merged DataFrame
print('For OUTPUTS')
display(merged_output_df)

For INPUTS


,guess,author,presynaptic_ID,counts,postsynaptic_ID,symbol
0,L1,94,720575940627244165,164,720575940639250741,L1
1,L5,94,720575940619166518,62,720575940639250741,L5
2,L3,94,720575940627317002,39,720575940639250741,L3
3,"OA-AL2b2; Busch et al., 2009; Tdc2-Gal4-positi...",2395,720575940633657631,24,720575940639250741,OA-AL2b2-R2
4,Proximal medulla 2; Pm2,17,720575940625500734,20,720575940639250741,Pm2
...,...,...,...,...,...,...
1500,Dm17,100,720575940634638562,1,720575940627669239,NaN
1501,None,None,720575940466786808,1,720575940627669239,NaN
1502,None,None,720575940609906198,1,720575940627669239,NaN
1503,Mi15,100,720575940623667470,1,720575940627669239,NaN


For OUTPUTS


,guess,author,postsynaptic_ID,counts,presynaptic_ID,postynaptic_ID,symbol
0,Pm4; Proximal medullary amacrine neuron 4; FBb...,392,720575940639071721,84,720575940622304895,720575940639071721,Pm4
1,Proximal medulla 2; Pm2,17,720575940618947329,47,720575940622304895,720575940618947329,Pm2
2,"Tm3; Transmedullary neuron 3; Fischbach, K.F.,...",392,720575940619319454,44,720575940622304895,720575940619319454,Tm3
3,Proximal medulla 1a; Pm1a,17,720575940633594669,44,720575940622304895,720575940633594669,Pm1a_Mi1
4,Proximal medulla 1a; Pm1a,17,720575940627351913,44,720575940622304895,720575940627351913,Pm1a_Mi1
...,...,...,...,...,...,...,...
9606,None,None,720575940524229045,1,720575940636425870,720575940524229045,NaN
9607,None,None,720575940524231093,1,720575940636425870,720575940524231093,NaN
9608,None,None,720575940409497429,1,720575940636425870,720575940409497429,NaN
9609,None,None,720575940409498709,1,720575940636425870,720575940409498709,NaN


### 6. Adding more useful information for later analysis

In [50]:
## For INPUTS

# Matcing data types
neuron_df['Updated_seg_id'] = neuron_df['Updated_seg_id'].astype(str)
merged_input_df['postsynaptic_ID'] = merged_input_df['postsynaptic_ID'].astype(str)

# Merging the DataFrames based on common values
merged_input_2_df = pd.merge(merged_input_df, neuron_df[['Updated_seg_id', 'optic_lobe_id','dorso-ventral']], left_on='postsynaptic_ID', right_on='Updated_seg_id', how='left')

# Drop the extra 'seg_id' column
merged_input_2_df.drop(columns=['Updated_seg_id'], inplace=True)

# Display the merged DataFrame
print('For INPUTS:')
display(merged_input_2_df)


## For OUTPUTS

# Matcing data types
neuron_df['Updated_seg_id'] = neuron_df['Updated_seg_id'].astype(str)
merged_output_df['presynaptic_ID'] = merged_output_df['presynaptic_ID'].astype(str)

# Merging the DataFrames based on common values
merged_output_2_df = pd.merge(merged_output_df, neuron_df[['Updated_seg_id', 'optic_lobe_id','dorso-ventral']], left_on='presynaptic_ID', right_on='Updated_seg_id', how='left')

# Drop the extra 'seg_id' column
merged_output_2_df.drop(columns=['Updated_seg_id'], inplace=True)

# Display the merged DataFrame
print('For OUTPUTS:')
display(merged_output_2_df)

For INPUTS:


,guess,author,presynaptic_ID,counts,postsynaptic_ID,symbol,optic_lobe_id,dorso-ventral
0,L1,94,720575940627244165,164,720575940639250741,L1,Mi1-R131,NaN
1,L5,94,720575940619166518,62,720575940639250741,L5,Mi1-R131,NaN
2,L3,94,720575940627317002,39,720575940639250741,L3,Mi1-R131,NaN
3,"OA-AL2b2; Busch et al., 2009; Tdc2-Gal4-positi...",2395,720575940633657631,24,720575940639250741,OA-AL2b2-R2,Mi1-R131,NaN
4,Proximal medulla 2; Pm2,17,720575940625500734,20,720575940639250741,Pm2,Mi1-R131,NaN
...,...,...,...,...,...,...,...,...
1500,Dm17,100,720575940634638562,1,720575940627669239,NaN,Mi1-R105,NaN
1501,None,None,720575940466786808,1,720575940627669239,NaN,Mi1-R105,NaN
1502,None,None,720575940609906198,1,720575940627669239,NaN,Mi1-R105,NaN
1503,Mi15,100,720575940623667470,1,720575940627669239,NaN,Mi1-R105,NaN


For OUTPUTS:


,guess,author,postsynaptic_ID,counts,presynaptic_ID,postynaptic_ID,symbol,optic_lobe_id,dorso-ventral
0,Pm4; Proximal medullary amacrine neuron 4; FBb...,392,720575940639071721,84,720575940622304895,720575940639071721,Pm4,Mi1-R664,NaN
1,Proximal medulla 2; Pm2,17,720575940618947329,47,720575940622304895,720575940618947329,Pm2,Mi1-R664,NaN
2,"Tm3; Transmedullary neuron 3; Fischbach, K.F.,...",392,720575940619319454,44,720575940622304895,720575940619319454,Tm3,Mi1-R664,NaN
3,Proximal medulla 1a; Pm1a,17,720575940633594669,44,720575940622304895,720575940633594669,Pm1a_Mi1,Mi1-R664,NaN
4,Proximal medulla 1a; Pm1a,17,720575940627351913,44,720575940622304895,720575940627351913,Pm1a_Mi1,Mi1-R664,NaN
...,...,...,...,...,...,...,...,...,...
9606,None,None,720575940524229045,1,720575940636425870,720575940524229045,NaN,Mi1-R671,NaN
9607,None,None,720575940524231093,1,720575940636425870,720575940524231093,NaN,Mi1-R671,NaN
9608,None,None,720575940409497429,1,720575940636425870,720575940409497429,NaN,Mi1-R671,NaN
9609,None,None,720575940409498709,1,720575940636425870,720575940409498709,NaN,Mi1-R671,NaN


### Saving 

In [51]:
# Saving data in your computer
PC_disc = 'D'
outDir = f'{PC_disc}:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\Mi1_patches' # YOUR-PATH for saving excel file
save_excel_file = True

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

if save_excel_file: 
    ## Input count
    file_name = f'{neuron}_neurons_input_count_{_hemisphere}_{date_str}.xlsx'
    savePath = os.path.join(outDir, file_name)
    merged_input_2_df.to_excel(savePath, sheet_name='Buhmann synapses, inputs')
    
    ## Output count
    file_name = f'{neuron}_neurons_output_count_{_hemisphere}_{date_str}.xlsx'
    savePath = os.path.join(outDir, file_name)
    merged_output_2_df.to_excel(savePath, sheet_name='Buhmann synapses, outputs')